In [1]:
import sys
import config
import serial

sys.path.insert(0, './fsm')
import PfLog

dre = PfLog.dre

# This is to allow consecutive partial executions of the notebook
if False:
    if dre is not None:
        if dre.ser is not None:
            dre.ser.close()


cte_serial_port = config.cte_serial_port

if config.cte_verbose:
    print("Chosen serial port: " + cte_serial_port)

dre.ser = serial.Serial(
    port=cte_serial_port,
    baudrate=9600,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    bytesize=serial.EIGHTBITS,
    rtscts=False,
    dsrdtr=False,
    xonxoff=True
)


Chosen serial port: COM6:


In [2]:
# Gets a response from the Motors
def getMotorResponse():
    PfLog.getCtrlResponse()
    return PfLog.dre.command_rx_buf, PfLog.dre.command_rx_str


In [ ]:
from datetime import datetime
from collections import OrderedDict
from pyexcel_ods import save_data


# Configuration
period1 = 6   # The number of samples per row for period 1
period2 = 18  # The number of samples per row for period 1
valid_status_limit = 6  # maximum status code to store values in rows.  
                        #If the status is greater than this maximum, the value will be stored as "Invalid"

# Initialization of counter and vectors for storage of samples
i1 = 1
i2 = 1
i = 1
period1_status1 = []
period2_status1 = []
period1_data1 = []
period2_data1 = []
period1_status2 = []
period2_status2 = []
period1_data2 = []
period2_data2 = []

period1_max_rows = 4
period2_max_rows = 4
i1_row = 1
i2_row = 1
sheet_data1 = OrderedDict() # from collections import OrderedDict
sheet_data2 = OrderedDict() # from collections import OrderedDict
px_header = ["tstamp", "min st1", "max st1", "min v1", "max v1", "mean v1", "current v1", "min st2", "max st2", "min v2", "max v2", "mean v2", "current v2"]
p1_row_data = []
p2_row_data = []
p1_sheet_data = [px_header]
p2_sheet_data = [px_header]
while (i <= 100):
    resp,resp2 = getMotorResponse()
    #print(i1,":",i2," ",resp2)
    splitresp = resp2.split(',')
    #print(splitresp)
    status1 = int(splitresp[0].strip())
    value1 = float(splitresp[1].strip())
    status2 = int(splitresp[2].strip())
    value2 = float(splitresp[3].strip())
    period1_status1.append(status1)
    period2_status1.append(status1)
    period1_status2.append(status2)
    period2_status2.append(status2)
    if (status1 < valid_status_limit):
        period1_data1.append(value1)
        period2_data1.append(value1)

    if (status2 < valid_status_limit):
        period1_data2.append(value2)
        period2_data2.append(value2)
        
    
    print(i1,": ",status1," ",value1," ",status2," ",value2)
    print(period1_status1)
    i1 = i1 + 1
    i2 = i2 + 1
    if (i1 > period1):
        p1_row_data = [datetime.now().strftime("%Y-%m-%d-%H%M%S"), min(period1_status1),max(period1_status1)]
        if (len(period1_data1) > 0):
            p1_row_data += [min(period1_data1),max(period1_data1),sum(period1_data1)/len(period1_data1),period1_data2[-1]]
        else:
            p1_row_data += ["Invalid","Invalid","Invalid","Invalid"]

        print("period1 tstamp: ",p1_row_data[0])
        print("period1 min status1: ",p1_row_data[1])
        print("period1 max status1: ",p1_row_data[2])
        print("period1 min value1: ",p1_row_data[3])
        print("period1 max value1: ",p1_row_data[4])
        print("period1 mean value1: ",p1_row_data[5])            
            
        p1_row_data += [min(period1_status2),max(period1_status2)]
        if (len(period1_data2) > 0):
            p1_row_data += [min(period1_data2),max(period1_data2),sum(period1_data2)/len(period1_data2),period1_data2[-1]]
        else:
            p1_row_data += ["Invalid","Invalid","Invalid","Invalid"]
        
        print("period1 min status2: ",p1_row_data[6])
        print("period1 max status2: ",p1_row_data[7])
        print("period1 min value2: ",p1_row_data[8])
        print("period1 max value2: ",p1_row_data[9])
        print("period1 mean value2: ",p1_row_data[10])
        #sheet_data1.update({"Sheet 1": [[1, 2, 3], [4, 5, 6]]})

        i1 = 1
        period1_status1 = []
        period1_data1 = []
        period1_status2 = []
        period1_data2 = []
        i1_row = i1_row + 1
        p1_sheet_data += [p1_row_data]
        print(p1_sheet_data)
        if (i1_row > period1_max_rows):
            sheet_data1.update({"Data": p1_sheet_data})
            dt_string = datetime.now().strftime("%Y-%m-%d-%H%M%S")
            print("Closing period1 (current) file with tstamp " + dt_string)
            save_data("p1_"+dt_string+".ods", sheet_data1)
            sheet_data1 = OrderedDict()
            print("Opening period1 (new) file")
            i1_row = 1
            p1_sheet_data = [px_header]
    
    if (i2 > period2):
        print("period2 min status1: ",min(period2_status1))
        print("period2 max status1: ",max(period2_status1))
        if (len(period2_data1) > 0):
            print("period2 min value1: ",min(period2_data1))
            print("period2 max value1: ",max(period2_data1))
            print("period2 mean value1: ",sum(period2_data1)/len(period2_data1))
        else:
            print("period2 min value1: ","No valid data")
            print("period2 max value1: ","No valid data")
            print("period2 mean value1: ","No valid data")
        
        
        print("period2 min status2: ",min(period2_status2))
        print("period2 max status2: ",max(period2_status2))
        if (len(period2_data1) > 0):
            print("period2 min value2: ",min(period2_data2))
            print("period2 max value2: ",max(period2_data2))
            print("period2 mean value2: ",sum(period2_data2)/len(period2_data2))
        else:
            print("period2 min value2: ","No valid data")
            print("period2 max value2: ","No valid data")
            print("period2 mean value2: ","No valid data")
            
        #sheet_data1.update({"Sheet 1": [[1, 2, 3], [4, 5, 6]]})
        i2 = 1
        period2_status1 = []
        period2_data1 = []
        period2_status2 = []
        period2_data2 = []
        i2_row = i2_row + 1
        if (i2_row > period2_max_rows):
            sheet_data2.update({"Data": p2_sheet_data})
            dt_string = datetime.now().strftime("%Y-%m-%d-%H%M%S")
            print("Closing period2 (current) file with tstamp " + dt_string)
            save_data("p2_"+dt_string+".ods", sheet_data2)
            sheet_data2 = OrderedDict()            
            print("Opening period2 (new) file")
            i2_row = 1
            p2_sheet_data = [px_header]
        
    


1 :  5   0.02   5   0.02
[5]
2 :  5   0.02   5   0.02
[5, 5]
3 :  5   0.02   5   0.02
[5, 5, 5]
4 :  5   0.02   5   0.02
[5, 5, 5, 5]
5 :  5   0.02   5   0.02
[5, 5, 5, 5, 5]
6 :  5   0.02   5   0.02
[5, 5, 5, 5, 5, 5]
period1 tstamp:  2020-03-12-125349
period1 min status1:  5
period1 max status1:  5
period1 min value1:  0.02
period1 max value1:  0.02
period1 mean value1:  0.02
period1 min status2:  0.02
period1 max status2:  5
period1 min value2:  5
period1 max value2:  0.02
period1 mean value2:  0.02
[['tstamp', 'min st1', 'max st1', 'min v1', 'max v1', 'mean v1', 'current v1', 'min st2', 'max st2', 'min v2', 'max v2', 'mean v2', 'current v2'], ['2020-03-12-125349', 5, 5, 0.02, 0.02, 0.02, 0.02, 5, 5, 0.02, 0.02, 0.02, 0.02]]
1 :  5   0.02   5   0.02
[5]
2 :  5   0.02   5   0.02
[5, 5]
3 :  5   0.02   5   0.02
[5, 5, 5]
4 :  5   0.02   5   0.02
[5, 5, 5, 5]
5 :  5   0.02   5   0.02
[5, 5, 5, 5, 5]
6 :  5   0.02   5   0.02
[5, 5, 5, 5, 5, 5]
period1 tstamp:  2020-03-12-125353
period1 

1 :  5   0.02   5   0.02
[5]
2 :  5   0.02   5   0.02
[5, 5]
3 :  5   0.02   5   0.02
[5, 5, 5]
4 :  5   0.02   5   0.02
[5, 5, 5, 5]
5 :  5   0.02   5   0.02
[5, 5, 5, 5, 5]
6 :  5   0.02   5   0.02
[5, 5, 5, 5, 5, 5]
period1 tstamp:  2020-03-12-125440
period1 min status1:  5
period1 max status1:  5
period1 min value1:  0.02
period1 max value1:  0.02
period1 mean value1:  0.02
period1 min status2:  0.02
period1 max status2:  5
period1 min value2:  5
period1 max value2:  0.02
period1 mean value2:  0.02
[['tstamp', 'min st1', 'max st1', 'min v1', 'max v1', 'mean v1', 'current v1', 'min st2', 'max st2', 'min v2', 'max v2', 'mean v2', 'current v2'], ['2020-03-12-125434', 5, 5, 0.02, 0.02, 0.02, 0.02, 5, 5, 0.02, 0.02, 0.02, 0.02], ['2020-03-12-125440', 5, 5, 0.02, 0.02, 0.02, 0.02, 5, 5, 0.02, 0.02, 0.02, 0.02]]
1 :  5   0.02   5   0.02
[5]
2 :  5   0.02   5   0.02
[5, 5]
3 :  5   0.02   5   0.02
[5, 5, 5]
4 :  5   0.02   5   0.02
[5, 5, 5, 5]
5 :  5   0.02   5   0.02
[5, 5, 5, 5, 5]
6 : 